In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

import os
import datetime
import time
import json
import pickle
import os
from sklearn.feature_selection import mutual_info_classif
from collections import Counter

NAFILL = -999

In [2]:
with open("config_general.json", "r") as f:
    config_general = json.load(f)
folder_path = config_general["data_root_path"]
epis = config_general["epis"]
wks = config_general["wks"]

In [3]:
with open(folder_path + "raw_data_pool.pkl", "rb") as f:
    raw_data_pool = pickle.load(f)

In [11]:
config_specific_filename = "config_specific_cmu_two_non_overlap.json"

In [12]:
with open(config_specific_filename, "r") as f:
    config_specific = json.load(f)
print(config_specific)

{'label_focus': 'BDI_full', 'phaseI_name': 'data_phaseI_bdivalid', 'phaseII_name': 'data_phaseII_bdivalid', 'feature_col_epis_filename': 'feature_col_epis_cmu.json', 'feature_dis_col_epis_filename': 'feature_dis_col_epis_cmu.json', 'mutual_info_num_feature': 100, 'coorelation_filtering_threshold': 0.5}


In [13]:
focus_path = folder_path + "Selection_Group/" + config_specific["label_focus"] + "/"
if (not os.path.exists(focus_path)):
    os.mkdir(focus_path)

In [14]:
df_phaseI_data = raw_data_pool[config_specific["phaseI_name"]]["data"]
df_phaseI_label = raw_data_pool[config_specific["phaseI_name"]]["label"]

In [15]:
df_phaseI_data.shape

(14522, 2211)

In [12]:
with open(folder_path + config_specific["feature_col_epis_filename"],"r") as f:
    feature_col_epis = json.load(f)

In [13]:
df_wks_epis = {}
feature_original_wks_epis = {}
for wk in wks:
    if wk not in df_wks_epis:
        df_wks_epis[wk] = {}
        feature_original_wks_epis[wk] = {}
    for epi in epis:
        print(wk,epi)
        df_buf = df_phaseI_data[df_phaseI_data["wk"] == wk][["PID"] + feature_col_epis[epi]]
        if(df_buf.shape[1] == 1): continue
        df_buf_mean = df_buf.groupby("PID").mean()
        df_buf_mean.columns = ["mean_" + x + "_" + wk for x in df_buf_mean.columns]
        df_buf_std = df_buf.groupby("PID").std()
        df_buf_std.columns = ["std_" + x + "_" + wk for x in df_buf_std.columns]
        df_buf_mean_std = df_buf_mean.merge(df_buf_std, how = "outer", left_index = True, right_index = True)
        feature_original_wks_epis[wk][epi] = deepcopy(df_buf_mean_std.columns.tolist())
#         df_buf_mean_std = (df_buf_mean_std-df_buf_mean_std.mean())/df_buf_mean_std.std()
        df_wks_epis[wk][epi] = deepcopy(df_buf_mean_std)

wkdy mo
wkdy af
wkdy ev
wkdy ni
wkend mo
wkend af
wkend ev
wkend ni


# Feature selection

In [14]:
top_features_ig = {}
for wk in wks:
    top_features_ig[wk] = {}
    for epi in epis:
        Y = df_phaseI_label[~df_phaseI_label["BDI"].isna()].set_index("PID")["label"]
        X = df_wks_epis[wk][epi].loc[Y.index]
        ig = dict(zip(X.columns, mutual_info_classif(X.fillna(NAFILL), Y, discrete_features = False)))
        ig = [(k, ig[k]) for k in sorted(ig, key=ig.get, reverse = True)]
        top_info_gain = [x[0] for x in ig[:config_specific["mutual_info_num_feature"]]]
        top_features_ig[wk][epi] = top_info_gain
overlap_features = top_features_ig
overlap_features_remain_ = deepcopy(overlap_features)
for i in range(30):
    features_ig = {}
    for wk in wks:
        features_ig[wk] = {}
        for epi in epis:
            Y = df_phaseI_label[~df_phaseI_label["BDI"].isna()].set_index("PID")["label"]
            X = df_wks_epis[wk][epi].loc[Y.index]
            ig = dict(zip(X.columns, mutual_info_classif(X.fillna(NAFILL), Y, discrete_features = False)))
            ig = [(k, ig[k]) for k in sorted(ig, key=ig.get, reverse = True)]
            top_features_ig = [x[0] for x in ig[:config_specific["mutual_info_num_feature"]]]
            overlap_features_remain_[wk][epi] = list(set(overlap_features_remain_[wk][epi]).intersection(set(top_features_ig)))

    for wk in wks:
        for epi in epis:
            print(len(overlap_features_remain_[wk][epi]),end = " ")
    print()


71 71 67 74 60 69 67 72 
58 62 59 64 50 58 59 58 
54 56 57 57 46 51 54 50 
53 56 56 53 40 50 49 45 
52 56 56 51 38 49 47 43 
50 56 55 47 37 48 47 42 
49 55 55 46 35 47 44 42 
47 55 55 46 34 46 42 42 
47 55 55 44 33 44 42 41 
47 54 55 43 33 44 42 41 
47 54 53 43 33 43 42 40 
47 54 53 42 33 42 42 40 
47 53 53 42 33 42 41 40 
47 48 52 42 33 42 41 40 
47 48 51 42 33 42 41 40 
47 48 51 40 33 42 41 37 
47 48 51 40 33 41 41 37 
47 48 51 40 33 41 41 36 
47 48 50 39 33 41 41 36 
47 48 50 39 32 39 41 36 
47 48 50 39 32 39 41 36 
47 48 50 39 31 39 41 36 
47 48 50 39 31 39 41 36 
47 48 49 39 31 39 41 36 
47 48 49 39 30 36 41 36 
47 48 49 39 29 36 41 36 
47 48 49 39 29 36 40 36 
47 48 49 39 29 36 40 36 
47 48 49 39 28 36 40 36 
47 48 49 38 28 36 40 36 


In [15]:
with open(focus_path + "top_features_raw.json", "w") as f:
    json.dump(overlap_features_remain_,f)
# with open(focus_path + "top_features_raw_II2I.json", "w") as f:
#     json.dump(overlap_features_remain_,f)

In [20]:
with open(focus_path + "top_features_raw.json", "r") as f:
    overlap_features_remain_ = json.load(f)
# with open(focus_path + "top_features_raw_II2I.json", "r") as f:
#     overlap_features_remain_ = json.load(f)

In [27]:
top_features_raw_dis = {}
for wk in wks:
    top_features_raw_dis[wk] = {}
    for ep in epis:
        for f in overlap_features_remain_[wk][ep]:
            split_f = f.split("_")
            f_dis = "_".join(split_f[1:-2] + ["dis"] + split_f[-2:])
            if (ep not in top_features_raw_dis[wk]):
                top_features_raw_dis[wk][ep] = [f_dis]
            else:
                top_features_raw_dis[wk][ep].append(f_dis)
        top_features_raw_dis[wk][ep] = list(set(top_features_raw_dis[wk][ep]))
with open(focus_path + "top_features_raw_dis.json", "w") as f:
    json.dump(top_features_raw_dis,f)

In [17]:
overlap_features_remain_filtered = deepcopy(overlap_features_remain_)
for wk in wks:
    for epi in epis:
        print(wk,epi)
        df_buf = df_wks_epis[wk][epi][overlap_features_remain_filtered[wk][epi]]
        df_buf_corr = df_buf.corr()
        upper = df_buf_corr.where(np.triu(np.ones(df_buf_corr.shape), k=1).astype(np.bool)).abs()
        max_corr = upper.max().max()
        while (max_corr > config_specific["coorelation_filtering_threshold"]):
            row_idxmax = upper.max(axis = 1).idxmax()
            col_idxmax = upper.max(axis = 0).idxmax()
            row_cor_sum = df_buf_corr.abs()[row_idxmax].sum()
            col_cor_sum = df_buf_corr.abs()[col_idxmax].sum()
            if(row_cor_sum > col_cor_sum):
                overlap_features_remain_filtered[wk][epi].remove(row_idxmax)
                print("remove ", row_idxmax)
            else:
                overlap_features_remain_filtered[wk][epi].remove(col_idxmax)
                print("remove ", col_idxmax)
            df_buf = df_wks_epis[wk][epi][overlap_features_remain_filtered[wk][epi]]
            df_buf_corr = df_buf.corr()
            upper = df_buf_corr.where(np.triu(np.ones(df_buf_corr.shape), k=1).astype(np.bool)).abs()
            max_corr = upper.max().max()
#         plt.matshow(df_buf_corr)
#         plt.show()
        print(Counter([x.split("_")[2] for x in overlap_features_remain_filtered[wk][epi]]))


wkdy mo
remove  std_f_locMap_num_bouts_at_place_007_morning_wkdy
remove  mean_f_locMap_num_bouts_10min_or_more_at_place_007_morning_wkdy
remove  std_f_screen_min_len_minute_unlock_bout_morning_wkdy
remove  mean_f_locMap_mean_bout_at_place_007_morning_wkdy
remove  std_f_screen_std_len_minute_unlock_bout_morning_wkdy
remove  mean_f_locMap_min_bout_at_place_007_morning_wkdy
remove  std_f_locMap_num_bouts_30min_or_more_at_place_008_morning_wkdy
remove  std_f_locMap_minutes_at_place_007_morning_wkdy
remove  std_f_slp_max_length_bout_restless_morning_wkdy
remove  mean_f_locMap_minutes_at_place_007_morning_wkdy
remove  mean_f_locMap_num_bouts_20min_or_more_at_place_008_morning_wkdy
remove  mean_f_screen_max_len_minute_unlock_bout_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_007_morning_wkdy
remove  mean_f_locMap_minutes_at_place_006_morning_wkdy
remove  std_f_loc_location_entropy_morning_wkdy
remove  mean_f_screen_max_len_minute_interaction_bout_morning_wkdy
remove  std_f_loc_mean_le

In [18]:
overlap_features_remain = {}
for wk in wks:
    overlap_features_remain[wk] = {}
    for epi in epis:
        print(len(overlap_features_remain_filtered[wk][epi]), end = ",")
        overlap_features_remain[wk][epi] = deepcopy(list(set(["_".join(x.split("_")[1:]) for x in overlap_features_remain_filtered[wk][epi]])))
        print(len(overlap_features_remain[wk][epi]))

20,20
25,25
25,25
16,16
12,12
19,19
21,21
14,14


In [19]:
with open(focus_path + "top_features.json", "w") as f:
    json.dump(overlap_features_remain,f)
with open(focus_path + "top_features_with_aggregation.json", "w") as f:
    json.dump(overlap_features_remain_filtered,f)

# with open(focus_path + "top_features_II2I.json", "w") as f:
#     json.dump(overlap_features_remain,f)
# with open(focus_path + "top_features_with_aggregation_II2I.json", "w") as f:
#     json.dump(overlap_features_remain_filtered,f)

In [16]:
with open(focus_path + "top_features.json", "r") as f:
    overlap_features_remain = json.load(f)

In [17]:
top_features_dis = {}
for wk in wks:
    top_features_dis[wk] = {}
    for ep in epis:
        for f in overlap_features_remain[wk][ep]:
            split_f = f.split("_")
            f_dis = "_".join(split_f[:-2] + ["dis"] + split_f[-2:])
            if (ep not in top_features_dis[wk]):
                top_features_dis[wk][ep] = [f_dis]
            else:
                top_features_dis[wk][ep].append(f_dis)

In [18]:
with open(focus_path + "top_features_dis.json", "w") as f:
    json.dump(top_features_dis,f)

### Ablation study

In [17]:
overlap_features_remain_filtered = deepcopy(feature_original_wks_epis)
for wk in wks:
    for epi in epis:
        print(wk,epi)
        df_buf = df_wks_epis[wk][epi][overlap_features_remain_filtered[wk][epi]]
        df_buf_corr = df_buf.corr()
        upper = df_buf_corr.where(np.triu(np.ones(df_buf_corr.shape), k=1).astype(np.bool)).abs()
        max_corr = upper.max().max()
        while (max_corr > config_specific["coorelation_filtering_threshold"]):
            row_idxmax = upper.max(axis = 1).idxmax()
            col_idxmax = upper.max(axis = 0).idxmax()
            row_cor_sum = df_buf_corr.abs()[row_idxmax].sum()
            col_cor_sum = df_buf_corr.abs()[col_idxmax].sum()
            if(row_cor_sum > col_cor_sum):
                overlap_features_remain_filtered[wk][epi].remove(row_idxmax)
                print("remove ", row_idxmax)
            else:
                overlap_features_remain_filtered[wk][epi].remove(col_idxmax)
                print("remove ", col_idxmax)
            df_buf = df_wks_epis[wk][epi][overlap_features_remain_filtered[wk][epi]]
            df_buf_corr = df_buf.corr()
            upper = df_buf_corr.where(np.triu(np.ones(df_buf_corr.shape), k=1).astype(np.bool)).abs()
            max_corr = upper.max().max()
#         plt.matshow(df_buf_corr)
#         plt.show()
        print(Counter([x.split("_")[2] for x in overlap_features_remain_filtered[wk][epi]]))


wkdy mo
remove  std_f_locMap_std_bout_at_place_001_morning_wkdy
remove  std_f_locMap_num_bouts_30min_or_more_at_place_001_morning_wkdy
remove  std_f_locMap_percent_time_at_place_002_morning_wkdy
remove  std_f_locMap_minutes_at_place_002_morning_wkdy
remove  std_f_loc_time_at_cluster_3_local_morning_wkdy
remove  mean_f_loc_number_samples_location_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_001_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_003_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_004_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_005_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_006_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_007_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_008_morning_wkdy
remove  mean_f_locMap_num_bouts_at_place_outside_morning_wkdy
remove  mean_f_slp_max_length_bout_totalsleep_morning_wkdy
remove  mean_f_slp_min_length_bout_totalsleep_morning_wkdy
remove  mean_f_slp_num_samples_sleep_morning_wkdy
rem

In [18]:
overlap_features_remain = {}
for wk in wks:
    overlap_features_remain[wk] = {}
    for epi in epis:
        print(len(overlap_features_remain_filtered[wk][epi]), end = ",")
        overlap_features_remain[wk][epi] = deepcopy(list(set(["_".join(x.split("_")[1:]) for x in overlap_features_remain_filtered[wk][epi]])))
        print(len(overlap_features_remain[wk][epi]))

77,59
64,57
63,57
62,53
77,66
68,60
91,73
65,58


In [19]:
with open(focus_path + "top_features_ablation_mutinfo.json", "w") as f:
    json.dump(overlap_features_remain,f)
with open(focus_path + "top_features_with_aggregation_ablation_mutinfo.json", "w") as f:
    json.dump(overlap_features_remain_filtered,f)